# 🤖 Building Agentic Workflows

**Sequential tool invocation** enables language models to perform complex multi-step reasoning by chaining function calls together. This notebook demonstrates how to build agentic workflows that extract information, process it, and provide comprehensive summaries.

## 🎯 What You'll Learn

1. Sequential tool calling patterns
2. Multi-step reasoning workflows
3. Conditional tool execution
4. Error handling in agentic workflows
5. Building comprehensive summaries
6. State management across tool calls

---

## 📊 Project Overview

**Goal:** Build an intelligent invoice analysis agent that can extract data and perform currency conversions.

**Process:**
1. 📝 Extract structured data from invoice text
2. 🔍 Analyze financial information
3. 💱 Convert currencies when needed
4. 📋 Generate comprehensive summaries
5. 🤖 Chain multiple AI operations

---

## 🚀 Section 1: Setup and Installation


In [ ]:
%pip install json-repair

---

## 🔌 Section 2: Imports and Setup

### 📡 Importing Required Libraries

We'll import all necessary libraries for building our agentic workflow, including the Google GenAI client, Pydantic for data validation, and other utilities.


In [2]:
import os
from google import genai
from google.genai import types
from pydantic import BaseModel, Field, field_validator
from datetime import date, datetime
from enum import Enum
from dotenv import load_dotenv
import requests
from typing import Optional, Dict, Any

load_dotenv()

from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY_1')

MODEL_NAME = "gemini-2.5-flash-lite"
client = genai.Client(
    api_key=api_key
)

print("Setup complete!")


Setup complete!


---

## 📊 Section 3: Data Models

### 💱 Currency and Invoice Models

We'll define Pydantic models to structure our data and ensure type safety throughout the workflow. These models will handle currency conversions and invoice data extraction.


In [3]:
class CurrencyEnum(str, Enum):
    USD = "USD"
    EUR = "EUR"
    JPY = "JPY"
    GBP = "GBP"
    AUD = "AUD"
    CAD = "CAD"
    CHF = "CHF"
    CNY = "CNY"
    HKD = "HKD"
    NZD = "NZD"


class CurrencyConversion(BaseModel):
    original_total: float = Field(description="Original amount before conversion")
    original_currency: str = Field(description="Original currency code")
    converted_total: float = Field(description="Converted amount")
    converted_currency: str = Field(description="Target currency code")
    exchange_rate: float = Field(description="Exchange rate used for conversion")


class Invoice(BaseModel):
    invoice_id: str | None = Field(
        default=None,
        description="Unique invoice number or identifier"
    )
    invoice_date: date | None = Field(
        default=None,
        description="Date when the invoice was issued"
    )
    due_date: date | None = Field(
        default=None,
        description="Date by which payment is due"
    )
    supplier_name: str | None = Field(
        default=None,
        description="Name of the company or person providing services"
    )
    supplier_address: str | None = Field(
        default=None,
        description="Physical address of the supplier"
    )
    supplier_tax_id: str | None = Field(
        default=None,
        description="Tax identification number of the supplier"
    )
    receiver_name: str | None = Field(
        default=None,
        description="Name of the entity or person being billed"
    )
    receiver_address: str | None = Field(
        default=None,
        description="Physical address of the receiver/client"
    )
    receiver_tax_id: str | None = Field(
        default=None,
        description="Tax identification number of the receiver/client"
    )
    total_amount: float | None = Field(
        default=None,
        ge=0,
        description="Total amount due for the entire invoice"
    )
    currency: CurrencyEnum | None = Field(
        default=None,
        description="Currency in which the invoice is issued"
    )
    payment_terms: int | None = Field(
        default=None,
        description="Payment terms in days"
    )

    @field_validator('invoice_date', 'due_date', mode='before')
    def validate_date_format(cls, v):
        if v is None:
            return v

        if isinstance(v, date):
            return v

        if isinstance(v, datetime):
            return v.date()

        if isinstance(v, str):
            v = v.strip()

            if v == '':
                return None

            date_formats = [
                '%Y-%m-%d', '%Y/%m/%d', '%d-%m-%Y', '%d/%m/%Y',
                '%m-%d-%Y', '%m/%d/%Y', '%d.%m.%Y', '%Y.%m.%d',
                '%B %d, %Y', '%b %d, %Y', '%d %B %Y', '%d %b %Y', '%Y%m%d',
            ]

            for fmt in date_formats:
                try:
                    return datetime.strptime(v, fmt).date()
                except ValueError:
                    continue

            raise ValueError(f'Date format not recognized: {v}')

        raise ValueError(f'Invalid date type: {type(v)}')


class InvoiceAnalysisResult(BaseModel):
    """Result of invoice analysis with optional currency conversion"""
    extracted_invoice: Invoice = Field(description="Extracted invoice data")
    currency_conversion: Optional[CurrencyConversion] = Field(
        default=None,
        description="Currency conversion if total amount was provided"
    )
    analysis_summary: str = Field(description="Comprehensive analysis summary")
    conversion_performed: bool = Field(description="Whether currency conversion was performed")


---

## 🛠️ Section 4: Tool Implementation

### 💱 Currency Conversion Tool

This tool enables our agent to convert currencies using real-time exchange rates from the Frankfurter API. It's a critical component for international invoice analysis.


In [4]:
def convert_currency(amount: float, from_currency: str, to_currency: str) -> dict[str, float | str]:
    """Convert currency using Frankfurter API"""
    url = f"https://api.frankfurter.dev/v1/latest?from={from_currency}"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        if to_currency not in data['rates']:
            available_currencies = ', '.join(data['rates'].keys())
            raise Exception(f"Currency '{to_currency}' not found in available rates. Available: {available_currencies}")

        rate = data['rates'][to_currency]
        converted_amount = amount * rate

        return {
            "original_total": amount,
            "original_currency": from_currency,
            "converted_total": converted_amount,
            "converted_currency": to_currency,
            "exchange_rate": rate,
        }
    else:
        raise Exception(f"Failed to fetch conversion rates: {response.status_code}")


test_conversion = convert_currency(100, "USD", "EUR")
print("Currency conversion tool working!")
print(f"Test: 100 USD = {test_conversion['converted_total']:.2f} EUR")


Currency conversion tool working!
Test: 100 USD = 86.12 EUR


---

## 🤖 Section 5: Agentic Workflow Implementation

### 📋 Invoice Analysis Agent

The core of our system - an intelligent agent that can:
1. **Extract** structured data from unstructured invoice text
2. **Analyze** financial information and payment terms
3. **Convert** currencies when needed using external APIs
4. **Summarize** findings in a comprehensive report

### 🔍 How the Agent Works

The agent follows a multi-step process:

**Step 1: Data Extraction**
- Uses AI to parse invoice text and extract structured information
- Identifies key fields like amounts, dates, parties, and terms

**Step 2: Currency Analysis**
- Checks if currency conversion is needed
- Calls the currency conversion tool when appropriate
- Maintains both original and converted amounts

**Step 3: Comprehensive Analysis**
- Generates detailed summary of findings
- Provides recommendations and insights
- Combines all information into a structured result


In [16]:
def analyze_invoice_with_agent(
    invoice_text: str,
    target_currency: str = "USD",
    client: genai.Client = None
) -> InvoiceAnalysisResult:
    """
    Agentic workflow to extract invoice data and perform currency conversion if needed

    This function demonstrates an AI agent that:
    1. Extracts structured data from invoice text (sequential call #1)
    2. Automatically converts currencies when needed (sequential call #2)
    3. Provides a comprehensive analysis summary (sequential call #3)

    Uses conversation history as memory system across sequential calls
    """

    # Step 1: Initialize the AI client
    if client is None:
        client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

    # Step 2: Create the currency conversion tool for the AI agent
    convert_currency_declaration = types.FunctionDeclaration(
        name="convert_currency",
        description="Converts an amount from one currency to another using the Frankfurter API.",
        parameters={
            "type": "object",
            "properties": {
                "amount": {
                    "type": "number",
                    "description": "The amount to be converted."
                },
                "from_currency": {
                    "type": "string",
                    "description": "The currency code of the original currency (e.g., 'USD')."
                },
                "to_currency": {
                    "type": "string",
                    "description": "The currency code to convert to (e.g., 'EUR')."
                }
            },
            "required": ["amount", "from_currency", "to_currency"]
        }
    )

    # Step 3: Define system prompt for the complete workflow
    system_prompt = """
    You are an invoice analysis agent. Your tasks are:
    1. Extract structured invoice data from text
    2. If a total amount and currency are found, convert to target currency
    3. Provide comprehensive analysis summary in markdown table format

    Always use the convert_currency tool when you have amount and currency information.
    """

    # ====================================================================
    # SEQUENTIAL CALL #1: Structured Output - Extract Invoice Data
    # ====================================================================
    extraction_prompt = f"""
    Extract invoice information from the following text and return it as structured JSON.
    Focus on identifying: invoice ID, dates, supplier/receiver details, amounts, and currency.

    Invoice text:
    {invoice_text}

    Return only valid JSON matching the Invoice schema.
    """

    # Initialize conversation memory
    conversation_history = [types.Content(role="user", parts=[types.Part(text=extraction_prompt)])]

    extraction_response = client.models.generate_content(
        model=MODEL_NAME,
        contents=conversation_history,
        config=types.GenerateContentConfig(
            system_instruction=system_prompt,
            response_mime_type="application/json",
            response_schema=Invoice,
            temperature=0.01,
        )
    )

    # Parse the extracted invoice data
    extracted_invoice = Invoice.model_validate_json(extraction_response.text)

    # Add the extraction response to conversation history (memory)
    conversation_history.append(extraction_response.candidates[0].content)

    print("="*200)
    print("SEQUENTIAL CALL #1: Structured Output Complete")
    print("="*200)

    # ====================================================================
    # SEQUENTIAL CALL #2: Tool Usage - Currency Conversion (if needed)
    # ====================================================================
    currency_conversion = None
    conversion_performed = False

    # Only attempt conversion if we have amount and currency info
    if extracted_invoice.total_amount and extracted_invoice.currency:
        tools = types.Tool(function_declarations=[convert_currency_declaration])

        conversion_request = f"""
        Based on the invoice data extracted above, convert the total amount to {target_currency}.

        Use the convert_currency tool with:
        - amount: {extracted_invoice.total_amount}
        - from_currency: {extracted_invoice.currency}
        - to_currency: {target_currency}
        """

        # Add the conversion request to conversation history
        conversation_history.append(types.Content(role="user", parts=[types.Part(text=conversion_request)]))

        conversion_response = client.models.generate_content(
            model=MODEL_NAME,
            contents=conversation_history,
            config=types.GenerateContentConfig(
                system_instruction=system_prompt,
                tools=[tools],
                temperature=0.01,
            )
        )

        # Add the conversion response to conversation history
        conversation_history.append(conversion_response.candidates[0].content)

        # Check if the AI wants to call the currency conversion tool
        if conversion_response.candidates and conversion_response.candidates[0].content.parts:
            conversion_part = conversion_response.candidates[0].content.parts[0]

            if conversion_part.function_call and conversion_part.function_call.name == "convert_currency":
                try:
                    # Execute the currency conversion
                    conversion_result = convert_currency(**conversion_part.function_call.args)
                    currency_conversion = CurrencyConversion.model_validate(conversion_result)
                    conversion_performed = True

                    # Add the tool execution result to conversation history
                    function_response_part = types.Part.from_function_response(
                        name=conversion_part.function_call.name,
                        response=conversion_result,
                    )
                    conversation_history.append(types.Content(role="user", parts=[function_response_part]))

                except Exception as e:
                    print(f"Currency conversion failed: {e}")

        print("="*200)
        print("SEQUENTIAL CALL #2: Currency Conversion Complete")
        print("="*200)

    # ====================================================================
    # SEQUENTIAL CALL #3: Summarization - Final Analysis
    # ====================================================================
    # Now the AI has access to all previous context from conversation_history
    final_analysis_request = """
    Based on all the information extracted and processed above, provide a comprehensive analysis.

    Include in a markdown table format:
    1. Key invoice details (ID, dates, parties)
    2. Financial information (amounts, currency)
    3. Payment terms and due dates
    4. Currency conversion details (if applicable)

    Make it concise and professional in markdown table format.
    Do not include any code, JSON structures, or function calls.
    """

    # Add the summarization request to conversation history
    conversation_history.append(types.Content(role="user", parts=[types.Part(text=final_analysis_request)]))

    final_response = client.models.generate_content(
        model=MODEL_NAME,
        contents=conversation_history,  # Model has full memory of the conversation
        config=types.GenerateContentConfig(
            system_instruction=system_prompt,
            temperature=0.01,
        )
    )

    print("="*200)
    print("SEQUENTIAL CALL #3: Summarization Complete")
    print("="*200)

    # Return the complete analysis result
    return InvoiceAnalysisResult(
        extracted_invoice=extracted_invoice,
        currency_conversion=currency_conversion,
        analysis_summary=final_response.text,
        conversion_performed=conversion_performed
    )

print("Invoice analysis agent with memory system created!")

Invoice analysis agent with memory system created!


In [42]:
def print_invoice_analysis(result: InvoiceAnalysisResult) -> None:
    """Print invoice analysis results in a clean, formatted way."""

    print("=" * 80)
    print("INVOICE ANALYSIS RESULTS")
    print("=" * 80)

    # Invoice Details
    print("\nINVOICE DETAILS:")
    print("-" * 40)
    invoice = result.extracted_invoice

    if invoice.invoice_id:
        print(f"Invoice ID: {invoice.invoice_id}")
    if invoice.invoice_date:
        print(f"Invoice Date: {invoice.invoice_date}")
    if invoice.due_date:
        print(f"Due Date: {invoice.due_date}")
    if invoice.supplier_name:
        print(f"Supplier: {invoice.supplier_name}")
    if invoice.receiver_name:
        print(f"Receiver: {invoice.receiver_name}")
    if invoice.total_amount:
        print(f"Total Amount: {invoice.total_amount}")
    if invoice.currency:
        print(f"Currency: {invoice.currency}")
    if invoice.payment_terms:
        print(f"Payment Terms: {invoice.payment_terms} days")

    # Currency Conversion
    if result.conversion_performed and result.currency_conversion:
        print("\nCURRENCY CONVERSION:")
        print("-" * 40)
        conv = result.currency_conversion
        print(f"Original Amount: {conv.original_total} {conv.original_currency}")
        print(f"Converted Amount: {conv.converted_total:.2f} {conv.converted_currency}")
        print(f"Exchange Rate: {conv.exchange_rate:.4f}")
    else:
        print("\nCURRENCY CONVERSION:")
        print("-" * 40)
        print("No currency conversion performed")

    # Analysis Summary
    print("\nANALYSIS SUMMARY:")
    print("-" * 40)
    print(result.analysis_summary)

    print("\n" + "=" * 80)
    print("END OF ANALYSIS")
    print("=" * 80)


print("Invoice analysis printing function created!")


Invoice analysis printing function created!


### 🚀 Running the Agentic Workflow

Now let's execute our complete agentic workflow and see how it processes the invoice data, performs currency conversion, and generates a comprehensive analysis.


In [38]:
# Sample invoice text for testing
sample_invoice = """
INVOICE

Invoice Number: INV-2025-100
Date: January 15, 2025

From:
Tech Solutions Inc.
123 Business Street
San Francisco, CA 94105

To:
Acme Corporation
456 Market Street
New York, NY 10001

Items:
1. Web Development - 40 hours @ $150/hr = $6,000
2. Cloud Hosting - 1 month @ $500 = $500

Total: $6,500 USD
Payment Terms: Net 30 days
"""

results = analyze_invoice_with_agent(sample_invoice, "EUR", client)

SEQUENTIAL CALL #1: Structured Output Complete
SEQUENTIAL CALL #2: Currency Conversion Complete
SEQUENTIAL CALL #3: Summarization Complete


In [43]:
print_invoice_analysis(results)

INVOICE ANALYSIS RESULTS

INVOICE DETAILS:
----------------------------------------
Invoice ID: INV-2025-100
Invoice Date: 2025-01-15
Due Date: 2025-02-14
Supplier: Tech Solutions Inc.
Receiver: Acme Corporation
Total Amount: 6500.0
Currency: CurrencyEnum.USD
Payment Terms: 30 days

CURRENCY CONVERSION:
----------------------------------------
Original Amount: 6500.0 USD
Converted Amount: 5597.67 EUR
Exchange Rate: 0.8612

ANALYSIS SUMMARY:
----------------------------------------
## Invoice Analysis Summary

| Category              | Details                                                              |
| :-------------------- | :------------------------------------------------------------------- |
| **Invoice Details**   |                                                                      |
| Invoice ID            | INV-2025-100                                                         |
| Invoice Date          | January 15, 2025                                                     |


---

## ⚖️ Section 6: LLM as a Judge Implementation

### 📋 Content Quality Validator

The LLM judge is a quality assurance component that evaluates generated responses for format compliance, clarity, and overall quality before delivery.

**Purpose:**
- Ensures responses meet expected quality standards
- Validates markdown format compliance
- Provides automated quality scoring and feedback
- Catches formatting errors before user delivery

### 🔍 How the Judge Works

The judge follows a structured evaluation process:

**Step 1: Pydantic Schema Definition**

- Defines `EvaluationScores` model for individual criteria (0-10 scale)
  - Format Compliance: Validates markdown structure
  - Clarity: Assesses readability and understandability
  - Completeness: Checks for all required information
  - Professionalism: Evaluates presentation quality
  - Accuracy: Identifies errors or inconsistencies

- Defines `LlmJudgeEvaluation` model for complete assessment
  - Aggregates individual scores into total (0-100)
  - Provides pass/fail decision based on threshold
  - Includes detailed feedback, issues list, and strengths list

**Step 2: Evaluation Execution**

- Sends response to LLM with structured evaluation criteria
- Uses `response_schema` parameter for enforced Pydantic validation
- LLM scores each criterion objectively
- Returns validated result matching strict schema

**Step 3: Quality Decision**

- Calculates total score from individual criteria
- Determines pass/fail based on quality threshold
- Provides actionable feedback for improvements
- Lists specific issues to address and strengths to maintain

### 💡 Key Features

- **Type Safety**: Pydantic models ensure structure and validation
- **Comprehensive Scoring**: Multi-criteria evaluation (5 dimensions)
- **Structured Feedback**: Detailed explanation of quality assessment
- **Issue Detection**: Identifies specific problems and improvements
- **Strength Recognition**: Highlights what was done well
- **Automated Validation**: Enforces response quality standards

### 📊 Evaluation Criteria

| Criterion | Description | Scale |
|-----------|-------------|-------|
| Format Compliance | Proper markdown formatting | 0-10 |
| Clarity | Readability and understandability | 0-10 |
| Completeness | Presence of all expected elements | 0-10 |
| Professionalism | Quality of presentation | 0-10 |
| Accuracy | Absence of errors/inconsistencies | 0-10 |
| **Total Score** | Aggregate quality assessment | 0-100 |


In [27]:
from typing import List

class EvaluationScores(BaseModel):
    """Individual scores for different criteria"""
    format_compliance: int = Field(ge=0, le=10, description="Score for format compliance (0-10)")
    clarity: int = Field(ge=0, le=10, description="Score for clarity (0-10)")
    completeness: int = Field(ge=0, le=10, description="Score for completeness (0-10)")
    professionalism: int = Field(ge=0, le=10, description="Score for professionalism (0-10)")
    accuracy: int = Field(ge=0, le=10, description="Score for accuracy (0-10)")


class LlmJudgeEvaluation(BaseModel):
    """Comprehensive LLM judge evaluation result"""
    scores: EvaluationScores = Field(description="Detailed scoring breakdown")
    total_score: int = Field(ge=0, le=100, description="Total score out of 100")
    passed: bool = Field(description="Whether the response passed quality threshold")
    feedback: str = Field(description="Detailed explanation of strengths and weaknesses")
    issues: List[str] = Field(default=[], description="List of specific issues found")
    strengths: List[str] = Field(default=[], description="List of what was done well")


def llm_judge(
    response: str,
    expected_format: str = "markdown",
    client: genai.Client = None
) -> LlmJudgeEvaluation:
    """
    Comprehensive LLM judge that evaluates response quality.

    Args:
        response: The response to evaluate
        expected_format: Expected output format
        client: GenAI client instance

    Returns:
        LlmJudgeEvaluation with scoring breakdown and detailed feedback
    """

    if client is None:
        client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

    judge_prompt = f"""
    As an expert content evaluator, analyze this response and provide detailed scoring.

    Expected Format: {expected_format}

    Response to evaluate:
    {response}

    Evaluate and score (0-10 each) for:
    - Format Compliance: Does it follow {expected_format} properly?
    - Clarity: Is it clear and easy to understand?
    - Completeness: Are all expected elements present?
    - Professionalism: Is it well-presented?
    - Accuracy: Any errors or inconsistencies?

    Provide detailed feedback, list specific issues if any, and highlight strengths.
    """

    result = client.models.generate_content(
        model=MODEL_NAME,
        contents=judge_prompt,
        config=types.GenerateContentConfig(
            system_instruction="You are a rigorous content evaluator. Provide objective, detailed assessment.",
            temperature=0.1,
            response_mime_type="application/json",
            response_schema=LlmJudgeEvaluation
        )
    )

    evaluation = LlmJudgeEvaluation.model_validate_json(result.text)

    return evaluation


print("Comprehensive LLM judge with Pydantic schema created!")

Comprehensive LLM judge with Pydantic schema created!


In [35]:
print("="*200)
print("LLM JUDGE EVALUATION")
print("="*200)

evaluation = llm_judge(
    response=results,
    expected_format="markdonw",
    client=client
)
print(evaluation.model_dump_json(indent=2))

LLM JUDGE EVALUATION
{
  "scores": {
    "format_compliance": 10,
    "clarity": 10,
    "completeness": 10,
    "professionalism": 10,
    "accuracy": 10
  },
  "total_score": 100,
  "passed": true,
  "feedback": "The response successfully adhered to the requested markdown format, presenting the invoice analysis in a clear, well-organized table. All expected elements were present and accurately extracted. The presentation was professional and easy to understand. No issues were identified.",
  "issues": [],
  "strengths": [
    "Excellent adherence to markdown formatting.",
    "Clear and concise presentation of invoice data.",
    "Accurate extraction of all relevant invoice and conversion details.",
    "Professional and well-structured analysis summary."
  ]
}


**bold text**

---

## 📚 Summary

### ✨ Key Concepts Covered

1. **Sequential Tool Calling**: Chaining multiple AI operations in a logical sequence
2. **Data Extraction**: Converting unstructured text to structured data using AI
3. **External API Integration**: Using real-time currency conversion services
4. **Error Handling**: Managing failures gracefully in multi-step workflows
5. **State Management**: Maintaining context across multiple AI interactions
6. **Comprehensive Analysis**: Generating detailed summaries from extracted data
7. **LLM as a judge**: Using LLM as a judge

### 💡 Key Insights

- ✅ **Agentic workflows** enable complex multi-step reasoning beyond single AI calls
- ✅ **Tool integration** allows AI to interact with external services and APIs
- ✅ **Structured data** ensures reliable processing and validation
- ✅ **Error handling** is crucial for production-ready agentic systems
- ✅ **State management** maintains context across sequential operations
- ✅ **Comprehensive analysis** provides actionable insights from raw data

### 🎯 Real-World Applications

Agentic workflows are used in many production systems:

- 🔹 **Document Processing**: Extract and analyze invoices, contracts, receipts
- 🔹 **Financial Analysis**: Convert currencies, calculate totals, assess risk
- 🔹 **Customer Service**: Multi-step problem resolution with external data
- 🔹 **Research Automation**: Gather, process, and summarize information
- 🔹 **Content Generation**: Create comprehensive reports from multiple sources

### 📖 Next Steps

- 🔹 Add more sophisticated error handling and retry logic
- 🔹 Implement parallel tool execution for better performance
- 🔹 Add validation and quality checks for extracted data
- 🔹 Explore more complex multi-agent workflows
- 🔹 Integrate with databases and external systems

### 🚀 Advanced Features to Explore

- **Parallel Processing**: Execute multiple tools simultaneously
- **Conditional Logic**: Smart decision-making based on extracted data
- **Memory Systems**: Remember context across multiple sessions
- **Human-in-the-Loop**: Interactive workflows with human oversight
- **Monitoring**: Track performance and success rates

---

### 🎓 Congratulations!

You now understand how to build sophisticated agentic workflows that can extract information, process it with external tools, and provide comprehensive analysis - all through intelligent AI orchestration!
